In [ ]:
import pandas as pd
import numpy as np
import EvaluateModels as em
import boto3

## defining bucket
s3 = boto3.resource('s3')

bucket_name = 'gabriel-de-medeiros'

bucket = s3.Bucket(bucket_name)

## specifying file from the bucket
file_key = 'CleanTrainData.csv'
file_key2 = 'CleanTestData.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')


## Reading csv file, index_col = 0 makes the first column of the data to become the index of our pandas data frame
train_data = pd.read_csv(file_content_stream, index_col = 0)
test_data = pd.read_csv(file_content_stream2, index_col = 0)

train_data = train_data.reset_index(drop = True)
test_data = test_data.reset_index(drop = True)

train_data = train_data.select_dtypes(exclude=['object'])
test_data = test_data.select_dtypes(exclude=['object'])

train_data = train_data.dropna()
test_data = test_data.dropna()


X_train, Y_train = train_data.drop(columns = 'is_fraud'), train_data['is_fraud']


### REMOVE LATER
X = X_train.loc[0:100000]
Y = Y_train.loc[0:100000]

import Get_Variable_Importance as gvi


df_importance_columns = gvi.Importance(X,Y)


X_test = test_data[df_importance_columns.drop(columns = 'is_fraud').columns]
Y_test = test_data['is_fraud']

### REMOVE LATER
X_test = X_test.loc[0:100000]
Y_test = Y_test.loc[0:100000]

X_train = train_data[df_importance_columns.drop(columns = 'is_fraud').columns]
Y_train = train_data['is_fraud']


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Decision Trees:


100%|██████████| 8/8 [00:08<00:00,  1.08s/it]


Random Forest:


100%|██████████| 40/40 [37:44<00:00, 56.61s/it]


In [ ]:
## Decision Trees
DTC_results = em.DecisionTreesResults(X_test, X_train, Y_test, Y_train)
DTC_best_model = em.DecisionTreesBestModel(DTC_results)

DecisionTrees


100%|██████████| 8/8 [00:34<00:00,  4.36s/it]


In [ ]:
DTC_best_model

,max_depth,cut_off,accuracy,recall,Performance
70,10,0.10,0.907384,0.806977,0.85424
71,10,0.15,0.907384,0.806977,0.85424
72,10,0.20,0.907384,0.806977,0.85424
73,10,0.25,0.907384,0.806977,0.85424
74,10,0.30,0.907384,0.806977,0.85424
75,10,0.35,0.907384,0.806977,0.85424
76,10,0.40,0.907384,0.806977,0.85424
77,10,0.45,0.907384,0.806977,0.85424
78,10,0.50,0.907384,0.806977,0.85424
79,10,0.55,0.907384,0.806977,0.85424


In [ ]:
LR_results = em.LogisticRegressionResults(X_test, X_train, Y_test, Y_train)

LogisticRegression


100%|██████████| 10/10 [00:00<00:00, 22.63it/s]


In [ ]:
LR_results

,cut_off,accuracy,recall,Performance
0,0.10,0.06002,0.988372,0.113168
1,0.15,0.06002,0.988372,0.113168
2,0.20,0.06002,0.988372,0.113168
3,0.25,0.06002,0.988372,0.113168
4,0.30,0.06002,0.988372,0.113168
5,0.35,0.06002,0.988372,0.113168
6,0.40,0.06002,0.988372,0.113168
7,0.45,0.06002,0.988372,0.113168
8,0.50,0.06002,0.988372,0.113168
9,0.55,0.06002,0.988372,0.113168


In [ ]:
NN_results = em.NeuralNetworksResults(X_test, X_train, Y_test, Y_train)

NeuralNetworks


  0%|          | 0/18 [00:00<?, ?it/s]

[2021-11-19 18:06:49.934 ip-172-16-27-137:15849 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-19 18:06:50.021 ip-172-16-27-137:15849 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [ ]:
RF_results = em.RandomForestResults(X_test, X_train, Y_test, Y_train)
RF_best_model = em.DecisionTreesBestModel(RF_results)

In [ ]:
## SVC
SVC_results = em.SupportVectorMachineResults(X_test, X_train, Y_test, Y_train)
SVC_best_model = em.SvcBestModel(SVC_results)

In [ ]:
## AdaBoost Decision Trees
ADA_DTC = em.AdaBoostDecisionTreesResults(X_test, X_train, Y_test, Y_train, DTC_best_model)



In [ ]:
## AdaBoost SVC
ADA_SVC= em.AdaBoostSvmResults(X_test, X_train, Y_test, Y_train, SVC_best_model)



In [ ]:
# GradientBoosting
GBC_results = em.GradientBoostingResults(X_test, X_train, Y_test, Y_train)

In [ ]:




# DTC_results.to_csv('CreditCardsFraudDetection/Algorithm/DTC_results.csv', index = False)
# DTC_best_model.to_csv('CreditCardsFraudDetection/Algorithm/DTC_best_model.csv', index = False)
# RF_results.to_csv('CreditCardsFraudDetection/Algorithm/RF_results.csv', index = False)
# RF_best_model.to_csv('CreditCardsFraudDetection/Algorithm/RF_best_model.csv', index = False)
# NN_results.to_csv('CreditCardsFraudDetection/Algorithm/NN_results.csv', index = False)
# SVC_results.to_csv('CreditCardsFraudDetection/Algorithm/SVC_results.csv', index = False)
# SVC_best_model.to_csv('CreditCardsFraudDetection/Algorithm/SVC_best_model.csv', index = False)
# LR_results.to_csv('CreditCardsFraudDetection/Algorithm/LR_results.csv', index = False)
# ADA_DTC.to_csv('CreditCardsFraudDetection/Algorithm/ADA_DTC.csv', index = False)
# ADA_SVC.to_csv('CreditCardsFraudDetection/Algorithm/ADA_SVC.csv', index = False)
# GBC_results.to_csv('CreditCardsFraudDetection/Algorithm/GBC_results.csv', index = False)